## Step1: make  a list of images to read in 

In [ ]:
import os
import glob
 
basedir = 'vehicles/'
#Different folders represent different sourses for images e.g.. GTI, Kitty, generated by me
image_types = os.listdir(basedir)
cars = []
for imtype in image_types:
    cars.extend(glob.glob(basedir+imtype+'/*'))
    
print('Number of Vehicle Images found:', len(cars))
with open ('cars.txt', 'w') as f:
    for fn in cars:
        f.write(fn+'\n')
        
basedir = 'non-vehicles/'
image_types = os.listdir(basedir)
notcars = []
for imtype in image_types:
    notcars.extend(glob.glob(basedir+imtype+'/*'))
print('Number of Non-Vehicle Images found:', len(notcars))
with open("notcars.txt", 'w') as f:
    for fn in notcars:
        f.write(fn+'\n')
        

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False,
                     feature_vec=True):                        
    
    
     # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [ ]:
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

In [ ]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins=nbins )
    ghist = np.histogram(img[:,:,1], bins=nbins )
    bhist = np.histogram(img[:,:,2], bins=nbins)
    hist_features = np.concatenate((rhist[0],ghist[0], bhist[0]))
    return hist_features

In [ ]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)     
        
        if spatial_feat == True:
            # Apply bin_spatial() to get spatial color features
            spatial_features = bin_spatial(feature_image, size=spatial_size) 
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist() also with a color space option now
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                        orient, pix_per_cell, cell_per_block,
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],orient,
                               pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            file_features.append(hog_features)
         
         
         
        # Append the new feature vector to the features list
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features


In [ ]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


In [ ]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True, vis=False):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
         
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
         
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            if vis == True:
                
                hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=True, feature_vec=True)
            else:
                
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
         
        img_features.append(hog_features)
    if vis==True:
        return np.concatenate(img_features), hog_image
    else:
        return np.concatenate(img_features)


In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

In [ ]:
def visualize(fig, rows, cols, imgs, titles):
    for i, img in enumerate(imgs):
        plt.subplot(rows, cols, i+1)
        plt.title(i+1)
        img_dims = len(img.shape)
        if img_dims<3:
            plt.imshow(img,cmap='hot')
            plt.title(titles[i])
        else:
            plt.imshow(img)
            plt.title(titles[i])
    

In [ ]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

In [ ]:
%matplotlib inline

#Choose random car/non-car indices
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))

#Read in car/non-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])

#Define feature parameters
color_space = 'YCrCb'#Can be RGB, HSV, LUV,HLS,YUV,YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 #Can be 0,1,2, 'ALL'
spatial_size = (16, 16)
hist_bins = 16
spatial_feat = True
hist_feat = True
hog_feat = True


converted_car_image = convert_color(car_image)
car_ch1 = converted_car_image[:,:,0]
car_ch2 = converted_car_image[:,:,1]
car_ch3 = converted_car_image[:,:,2]

converted_notcar_image = convert_color(notcar_image)
notcar_ch1 = converted_notcar_image[:,:,0]
notcar_ch2 = converted_notcar_image[:,:,1]
notcar_ch3 = converted_notcar_image[:,:,2]


car_features, car_hog_image = single_img_features(car_image,
             color_space=color_space, spatial_size = spatial_size,
             hist_bins = hist_bins, orient= orient, pix_per_cell=pix_per_cell,
             cell_per_block=cell_per_block,hog_channel=hog_channel,
             spatial_feat=spatial_feat,hist_feat=hist_feat, hog_feat=hog_feat,
             vis = True)

notcar_features, notcar_hog_image = single_img_features(notcar_image,
             color_space=color_space, spatial_size = spatial_size,
             hist_bins = hist_bins, orient= orient, pix_per_cell=pix_per_cell,
             cell_per_block=cell_per_block,hog_channel=hog_channel,
             spatial_feat=spatial_feat,hist_feat=hist_feat, hog_feat=hog_feat,
             vis = True)


car_ch1_features = cv2.resize(car_ch1, spatial_size)
car_ch2_features = cv2.resize(car_ch2, spatial_size)
car_ch3_features = cv2.resize(car_ch3, spatial_size)

notcar_ch1_features = cv2.resize(notcar_ch1, spatial_size)
notcar_ch2_features = cv2.resize(notcar_ch2, spatial_size)
notcar_ch3_features = cv2.resize(notcar_ch3, spatial_size)

def show_images(image1, image2, image3, image4,  image1_exp="Image 1", image2_exp="Image 2", image3_exp="Image 3", image4_exp="Image 4"):
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image1)
    ax1.set_title(image1_exp, fontsize=20)
    ax2.imshow(image2)
    ax2.set_title(image2_exp, fontsize=20)
    ax3.imshow(image3)
    ax3.set_title(image3_exp, fontsize=20)
    ax4.imshow(image4)
    ax4.set_title(image4_exp, fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

show_images(car_ch1, car_hog_image, notcar_ch1, notcar_hog_image, "Car ch 1", "Car ch 1 HOG", "Not Car ch 1", "Not Car ch 1 HOG")    
show_images(car_ch1, car_ch1_features, notcar_ch1, notcar_ch1_features, "Car ch 1", "Car ch 1 features", "Not Car ch 1", "Not Car ch 1 features")    
show_images(car_ch2, car_ch2_features, notcar_ch2, notcar_ch2_features, "Car ch 2", "Car ch 2 features", "Not Car ch 2", "Not Car ch 2 features")    
show_images(car_ch3, car_ch3_features, notcar_ch3, notcar_ch3_features, "Car ch 3", "Car ch 3 features", "Not Car ch 3", "Not Car ch 3 features")    



In [ ]:
class TrainedModel:
    svc = None
    X_scaler = None
    def __init__(self, svc, X_scaler):
        self.svc = svc
        self.X_scaler = X_scaler        

In [ ]:
#Define feature parameters
color_space = 'YCrCb'#Can be RGB, HSV, LUV,HLS,YUV,YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 #Can be 0,1,2, 'ALL'
spatial_size = (16, 16)
hist_bins = 16
spatial_feat = True
hist_feat = True
hog_feat = True


# Train the clasifier

In [ ]:
import os.path

if os.path.isfile('my_dumped_classifier.p'):
    print("file found skipping training")
else:
    t = time.time()
    n_samples = 1000
    random_idxs = np.random.randint(0,len(cars), n_samples)
    test_cars = cars#np.array(cars)[random_idxs]
    test_notcars = notcars #np.array(notcars)[random_idxs]

    car_features = extract_features(test_cars, color_space=color_space, spatial_size = spatial_size,
                 hist_bins = hist_bins, orient= orient, pix_per_cell=pix_per_cell,
                 cell_per_block=cell_per_block,hog_channel=hog_channel,
                 spatial_feat=spatial_feat,hist_feat=hist_feat, hog_feat=hog_feat)

    notcar_features = extract_features(test_notcars, color_space=color_space, spatial_size = spatial_size,
                 hist_bins = hist_bins, orient= orient, pix_per_cell=pix_per_cell,
                 cell_per_block=cell_per_block,hog_channel=hog_channel,
                 spatial_feat=spatial_feat,hist_feat=hist_feat, hog_feat=hog_feat)

    print(time.time() - t, "Seconds to compute features..")


    print(car_features[0])

    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)

    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    rand_state = np.random.randint(0,100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.1, random_state=rand_state)

    print("Using:", orient, "orientations", pix_per_cell, "pixels per cell", 
         cell_per_block, "cells per block", hist_bins, "histogram bins, and",
         spatial_size, "spatial sampling")
    print("Feature vector lenght:", len(X_train[0]))
    svc = LinearSVC()
    t = time.time()
    svc.fit(X_train, y_train)
    print(round(time.time()-t,2),"Seconds to train SVC...")
    print("Test accuracy of svc = ", round(svc.score(X_test, y_test), 4))

    import pickle
    # save the classifier
    with open('my_dumped_classifier.p', 'wb') as fid:
        pickle.dump(TrainedModel(svc,X_scaler), fid) 

In [ ]:
import pickle
with open('my_dumped_classifier.p', 'rb') as fid:
    
    trained_model = pickle.load(fid)
    svc = trained_model.svc
    X_scaler = trained_model.X_scaler

In [ ]:

def add_heat(heatmap, bbox_list, amount = 1):
    # box is in format ((x1,y1),(x2,y2))
    for box in bbox_list:
        
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += amount
    return heatmap

In [ ]:
def find_cars(img, ystart, ystop, scale, cspace, hog_channel, svc, X_scaler, orient, pix_per_cell, cell_per_block, 
              spatial_size, hist_bins, cells_per_step = 2, debug_show_all = False):
    img_boxes = []
    draw_img = np.copy(img)
    #make a heatmap of zeros
    heatmap = np.zeros_like(img[:,:,0])
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
        
    if cspace != 'RGB':
        if cspace == 'HSV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    else: ctrans_tosearch = np.copy(image)   
       
    
    if scale != 1:
        
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale),  np.int(imshape[0]/scale)) )
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    
    #define blocks and steps as above
    nxblocks = (ch1.shape[1]//pix_per_cell) - 1
    nyblocks = (ch1.shape[0] //pix_per_cell) - 1
    nfeat_per_block = orient*cell_per_block**2
    window = 64
    nblocks_per_window = (window // pix_per_cell) - 1
    
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    #compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec = False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec = False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec = False)
    
    count = 0
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            count+=1
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step 
            #extract hog for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_ch_arr = [hog_feat1, hog_feat2, hog_feat3]
            
            if hog_channel == 'ALL':                
                hog_features = np.hstack((hog_feat1,hog_feat2,hog_feat3))
            else:
                hog_features = hog_ch_arr[hog_channel]
                
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
            #extact the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
            
            #get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            #scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1,-1))
            test_predictions = svc.predict(test_features)
                        
            if test_predictions == 1 or debug_show_all:                
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)                            
                box = ((xbox_left, ytop_draw + ystart), (xbox_left + win_draw, ytop_draw + win_draw + ystart))         
                cv2.rectangle(draw_img, box[0], box[1], (0,0,255), 4)
                img_boxes.append(box)
                
                #heatmap[ytop_draw+ystart:ytop_draw+win_draw+ystart, xbox_left:xbox_left+win_draw]+=1
            
    return img_boxes   

In [ ]:
test_img = mpimg.imread('./test_images/test1.jpg')

detections = []       

ystart = 400
ystop = 500
scale = 1.0
detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                       orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,cells_per_step=1, debug_show_all=True))

ystart = 400
ystop = 656
scale = 1.5
detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                       orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, cells_per_step=1,  debug_show_all=True))

ystart = 400
ystop = 656
scale = 2.0
detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                       orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,  debug_show_all=True))

test_img_rects = draw_boxes(test_img, detections, color=(255,0,0), thick=2)
fig = plt.figure(figsize=(10,5))    
plt.imshow(test_img_rects)
fig.savefig('all_rects_example')

In [ ]:
test_imgs =  []
indices = [1,4,5,6]
for indx in indices:
    file_name = './test_images/test{0}.jpg'.format(str(indx))
    test_img = mpimg.imread(file_name)

    detections = []       

    ystart = 400
    ystop = 500
    scale = 1.0
    detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,cells_per_step=1))

    ystart = 400
    ystop = 656
    scale = 1.5
    detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, cells_per_step=1))

    ystart = 400
    ystop = 656
    scale = 2.0
    detections.extend(find_cars(test_img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins))    

    test_img_rects = draw_boxes(test_img, detections, color=(255,0,0), thick=2)
    test_imgs.append(test_img_rects)
    
fig = plt.figure(figsize=(10,10))    
visualize(fig, 2,2, test_imgs, ['image1', 'image2', 'image3', 'image4'])
fig.savefig('detections_example')

In [ ]:
from scipy.ndimage.measurements import label

def apply_threshold(heatmap, threshold):
    heatmap[heatmap<=threshold] = 0
    return heatmap

def draw_labeled_bboxes(img, labels):
    #iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        #find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        #identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    return img

In [ ]:
class Detection_Buffer():
    
    def __init__(self):
        self.buffer = []
        
    buffer_len = 10
    heat_map = None
    
    def get_cur_buf_len(self):
        return len(self.buffer)
    
    def is_buffer_full(self):
        return self.buffer_len == self.get_cur_buf_len()
    
    def add_frame_detections(self, detections):
        self.heat_map = add_heat(self.heat_map, detections)
        
        self.buffer.append(detections)
        
        if len(self.buffer)> self.buffer_len:
            self.heat_map = add_heat(self.heat_map, self.buffer[0], -1)
            self.buffer = self.buffer[len(self.buffer) - self.buffer_len:]    
            

In [ ]:
detection_buffer = Detection_Buffer()
 

In [ ]:
def process_image(img):
    if detection_buffer.heat_map is None:
        detection_buffer.heat_map = np.zeros_like(img[:,:,0], dtype = np.int16)
    
    detections = []       
    
    ystart = 400
    ystop = 500
    scale = 1.0
    detections.extend(find_cars(img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,cells_per_step=1))
    
    ystart = 400
    ystop = 656
    scale = 1.5
    detections.extend(find_cars(img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, cells_per_step=1))
    
    ystart = 400
    ystop = 656
    scale = 2.0
    detections.extend(find_cars(img, ystart, ystop, scale, color_space, hog_channel, svc, X_scaler, 
                           orient, pix_per_cell, cell_per_block, spatial_size, hist_bins))
    
     
    
    detection_buffer.add_frame_detections(detections)
           
    threshold = 2
    
    heat_map = apply_threshold(detection_buffer.heat_map, threshold)
    labels = label(heat_map)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    
    draw_cur_frame_rects = False
    
    if draw_cur_frame_rects:
        draw_img = draw_boxes(draw_img, detections, (255, 0, 0))
       
    global create_example_images 
    
    
    if create_example_images:
        
        if detection_buffer.is_buffer_full():
            create_example_images = False
            
            
            plot_images = []
            for frame_rectangles in detection_buffer.buffer:
                bbox_image = draw_boxes(np.copy(img), frame_rectangles)
                plot_images.append(bbox_image)
                
                frame_heat = np.zeros_like(img[:,:,0], dtype = np.int16)
                frame_heat = add_heat(frame_heat, frame_rectangles)
                plot_images.append(frame_heat)  
                
            
            out_titles = [str(i) for i in range(detection_buffer.buffer_len*2)]
             
            fig = plt.figure(figsize=(10,20))
            visualize(fig, detection_buffer.buffer_len, 2, plot_images, out_titles)
            fig.savefig('bboxes_and_heat')
            
             
            fig = plt.figure(figsize=(10,5))
            plt.imshow(labels[0], cmap='gray')
            fig.savefig('labels_map')
            
            fig = plt.figure(figsize=(10,5))
            out_boxes_img = draw_labeled_bboxes(np.copy(img), labels)
            plt.imshow(out_boxes_img)
            fig.savefig('output_bboxes')
            
        
    return draw_img

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import  HTML

create_example_images = False
detection_buffer = Detection_Buffer()

test_output = 'test.mp4'
#clip = VideoFileClip("test_video.mp4")
#clip = VideoFileClip("project_video.mp4")

start = 39 
clip_duration = .5

clip = VideoFileClip("project_video.mp4").subclip(start, start + clip_duration )
test_clip = clip.fl_image(process_image)
#%time
test_clip.write_videofile(test_output, audio = False)

clip.reader.close()
clip.audio.reader.close_proc()

In [ ]:
HTML("""
<video width="960" height="540" controls>
    <source src="{0}">
</video>
""".format(test_output))

In [ ]:
class Vehicle():
    def __init__(self):
        self.detected = False
        self.n_detections = 0
        self.n_nondetections = 0
        self.xpixels = None
        self.ypixels = None
        self.recent_xfitted = []
        self.bestx = None
        self.recent_yfitted = []
        self.besty = None
        self.recent_yfitted = []
        self.besty = None
        self.recent_wfitted = []
        self.bestw = None
        self.recent_hfitted = []
        self.besth = None